In [1]:
def translate(param, G, T, Isc_NOTC, Isc_STC=None, alpha=None):
    kb = 1.38*10**(-23) #Boltzmann constant
    q = 1.6*10**(-19)
    Eg = 1.1
    Rs, Rsh, Isat, Iph, eta = param
    
    if not Isc_STC:
        Isc_STC = Iph
    
    Rs_n = Rs                    # No change in series resistance
    if not alpha:                # If alpha not given
        alpha = Isc_STC*0.05/100
    K = np.log((Isc_NOTC-alpha*(45-25))/Isc_STC)/np.log(800/1000) # (T_NOTC - T_STC), (G_NOTC/G_STC)
    Isc_n = ((G/1000)**K) * (Isc_STC + alpha*(T-25))  # New Isc (Used later for Iph calc) 
    Rsh_n = (1000/G)*Rsh
    Iph_n = Isc_n*(1+Rs_n/Rsh_n)
    Isat_n = Isat*(((T+273)/(25+273))**3)*np.exp((Eg*q)/(eta*kb) * (1/(273+25) - 1/(T+273)))
    return (Rs_n, Rsh_n, Isat_n, Iph_n, eta)

In [2]:
import pandas as pd

def generate_IV(module_param, Temp, datapoints, Ncell, Voc=None, Isc=None):
    """
    This function will generate I-V data points given all the 5 diode parameters 
    
    Input  : 5 parameters: Rs, Rsh, Isat, Iph, eta
             Voc, Temperature(Celcius), N(Number of data points), Ncell
             
    Output : Returns a dictionary of I-V data (size = N) and metadata (data, metadata)
    
    This function will spit out I-V data points. 
    
    ** This auto generates Voc and Isc also if not provided **
    **Temperature is just taken to calculate Vt**
    
    """
    kb = 1.38*10**(-23) #Boltzmann constant
    q = 1.6*10**(-19)
    Vt = kb*(Temp+273)/q
    
    Rs, Rsh, Isat, Iph, eta = module_param
    
    # If the Voc argument is not passed explicitly, this will calculate Voc also from the 5 parameters
    if not Voc:             # If Voc=None
        Voc_implicit_eqn = lambda voc: Iph - Isat*(np.exp((voc) / (eta*Ncell*Vt))-1) - (voc)/Rsh # i=0 @Voc
        Vocr = root(Voc_implicit_eqn,50)
        Voc = np.round(Vocr.x[0], 2)
        #print ('Auto generated Voc is :', Voc)
    
    # If the Isc argument is not passed explicitly, this will calculate Isc also from the 5 parameters
    if not Isc:             # If Voc=None
        Isc_implicit_eqn = lambda isc: -isc + Iph - Isat*(np.exp((isc*Rs) / (eta*Ncell*Vt))-1) - (isc*Rs)/Rsh # v=0 @Isc
        Isc = np.round(root(Isc_implicit_eqn,50).x[0], 2)
        #print ('Auto generated Isc is :', Isc)    
    
    # Now solving the implicit equation of the solar cell 
    I_implicit_equn = lambda i: -i + Iph - Isat*(np.exp((v+i*Rs) / (eta*Ncell*Vt))-1) - (v+i*Rs)/Rsh
    V = np.linspace(0,Voc,datapoints)  #Generating the Voltage vector
    I = []
    for v in V:                        # For each value of Voltage, current is calculated
        ior = root(I_implicit_equn,10)
        io = ior.x[0]
        I.append(io)
    
    I = np.round(I, 2)                 # Rounding upto 2 places
    V = np.round(V, 2)
    
    ## Now calculating Pmax, FF, Im, Vm
    df = pd.DataFrame(dict(V=V, I=I))
    df['P'] = df['I']*df['V']
    values = df.loc[df['P'].idxmax()]
    Vm, Im, Pmax = values
    FF = np.round(Pmax/(Voc*Isc),2)
    
    #metadata = dict(Voc=Voc, Isc=Isc, Pmax=np.round(Pmax,2), Vmp=Vm, Imp=Im, FF=FF) 
    metadata = dict(Voc=[Voc], Isc=[Isc], Pmax=[np.round(Pmax,2)], Vmp=[Vm], Imp=[Im], FF=[FF]) 
    data = dict(x=V, y=I)
    
    return (data, metadata)              # Returning a dictionary of data and metadata

In [5]:
from bokeh.models import Slider, CustomJS, Div
from bokeh.models.widgets import Button, TextInput, DataTable, TableColumn
from bokeh.layouts import row, column, gridplot
from bokeh.plotting import Figure, output_notebook, show, ColumnDataSource, output_file
import pandas as pd
from scipy.optimize import root
import numpy as np
from os.path import dirname, join

output_notebook()

def plot_iv(doc):
    
    diode_param = (0.3551, 20051.4912, 2.9778e-10, 9.2501, 1.0266)
    data_pair_STC = generate_IV(diode_param, 25, 100, 72)
    source_STC = ColumnDataSource(data=data_pair_STC[0])
    
    #source_STC.update(Temp=[25], Irradiance=[])
    
    metasource_STC = ColumnDataSource(data=data_pair_STC[1])
    col = [
        TableColumn(field="Voc", title="Voc"),
        TableColumn(field="Isc", title="Isc"),
        TableColumn(field="Pmax", title="Pmax"),
        TableColumn(field="Vmp", title="Vmp"),
        TableColumn(field="Imp", title="Imp"),
        TableColumn(field="FF", title="Fill Factor"),
    ]
    meta_table_STC = DataTable(source=metasource_STC, columns=col, height=80, index_header='', fit_columns=True)
    
    params_new = translate(diode_param, 600, 40, 7.5, 9.25, alpha=None)
    data_pair_new = generate_IV(params_new, 40, 100, 72)
    source_GT = ColumnDataSource(data=data_pair_new[0])
    metasource_GT = ColumnDataSource(data=data_pair_new[1])
    Temp, Irrad = 40, 600
    meta_table_GT = DataTable(source=metasource_GT, columns=col, height=80, index_header='')
    
    plot = Figure(title='I-V plot @STC and given conditions', tools="hover", tooltips="@x: @y", height=500,)# y_range = (-1, 10), x_range = (0,60))
    plot.xaxis.axis_label = 'Voltage (V)'
    plot.yaxis.axis_label = 'Current (I)'
    plot.scatter('x', 'y', legend='STC', source=source_STC, line_width=3, line_color='green', line_alpha=0.6, name='STC Plot')
    plot.scatter('x', 'y', legend='Translated',source=source_GT, line_width=3,line_color='blue', line_alpha=0.6, name='Translated Plot')
    
    
    Irrad_input = TextInput(value='600', title='Irradiance(W/m^2)', width=150)
    Temp_input = TextInput(value='40', title='Temperature (Celcius)', width=150)
    Rs_input = TextInput(value='0.35', title='Series Resistance', width=150)
    Rsh_input = TextInput(value='20051', title='Shunt Resistance', width=150)
    Isat_input = TextInput(value='2.97e-10', title='Isat', width=100)
    Iph_input = TextInput(value='9.25', title='Iph', width=100)
    eta_input = TextInput(value='1.02', title='Eta (Ideality Factor)', width=100)
    Isc_NOTC_input = TextInput(value='7.47', title='I_sc at NOTC(G=800, T=45C)', width=180)
    Isc_STC_input = TextInput(value='9.25', title='I_sc at STC(G=1000, T=25C)', width=160)
    alpha_input = TextInput(value='0.005', title='Alpha (Temp. Coeff Isc)', width=150)
    datasize_input = TextInput(value='100', title='Data Size')
    submit = Button(label='Submit', button_type='success')
    dwnld = Button(label='Download (csv)',)
    
    
    div_STC = Div(text=""" <b><font color="green">DataTable @ STC</font></b> """)
    div_GT = Div(text=""" <b><font color="blue">DataTable @ given conditions</font></b> """)
    div_param = Div(text=""" <h2><font color="green">5 Module Parameters @STC</font></h2> """)
    div_condition = Div(text=""" <h2><font color="blue">Conditions</font></h2> """)
    
    def get_inputs():
        return (float(Irrad_input.value) ,float(Temp_input.value), float(alpha_input.value), 
                float(Isc_STC_input.value), float(Isc_NOTC_input.value),
                (float(Rs_input.value) ,float(Rsh_input.value) ,float(Isat_input.value) ,float(Iph_input.value) ,float(eta_input.value)),
                int(datasize_input.value))
    
    def update_plot():
        G, T, alpha, Isc_STC, Isc_NOTC, params, size = get_inputs()
        global Temp, Irrad
        Temp, Irrad = T, G
        data_pair_STC = generate_IV(params, 25, size, 72)
        source_STC.data = data_pair_STC[0]
        metasource_STC.data = data_pair_STC[1]
        
#         print (f'Isc_STC is {Isc_STC}')
        params_new = translate(params, G, T, Isc_NOTC, Isc_STC, alpha)
        print ()
        data_pair_new = generate_IV(params_new, T, size, 72)
        source_GT.data = data_pair_new[0]
        metasource_GT.data = data_pair_new[1]

    def download():
        df = pd.DataFrame(source_STC.data)
        df2 = pd.DataFrame(source_GT.data)
        global Temp, Irrad
        df2.to_csv(f'{Irrad}Wpm^2_{Temp}C-ivdata_GT.csv')
        df.to_csv('iv-data_STC.csv')
    
    submit.on_click(update_plot)
    dwnld.on_click(download)
    param_column = column(row(Rs_input, Rsh_input), row(Isat_input, Iph_input, eta_input))
    layout = row(plot, column(div_condition, row(Irrad_input, Temp_input), alpha_input, row(Isc_NOTC_input, Isc_STC_input), column(div_param, param_column), datasize_input, submit, dwnld),)
    doc.add_root(column(layout, column(div_STC,meta_table_STC), column(div_GT, meta_table_GT)))

show(plot_iv)

Loading BokehJS ...